In [7]:
%load_ext autoreload
%autoreload 2

import os
import gpflow
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
mpl.rcParams['pgf.rcfonts'] = False

os.environ["CUDA_VISIBLE_DEVICES"] = ''

from gp_svgp import GP_MNIST_SVGP
from mnist_cnn import CNN_MNIST
from mnist_svm import SVM_MNIST
from hybrid_mnist import Hybrid_MNIST
from mnist_preprocessing import get_mnist_data

#from plotting import Plotter

In [8]:
data = {
    'standard': {
        'accuracy': {}
    }
}

In [9]:
# Standard MNIST test data
((_, mnist_test_imgs), (ys_train_onehot, ys_test_onehot)) = get_mnist_data()
mnist_train_features = np.genfromtxt("../data/mnist_train_features.csv", delimiter=",")
mnist_test_features = np.genfromtxt("../data/mnist_test_features.csv", delimiter=",")

ys_train = np.argmax(ys_train_onehot, axis=1)
ys_test = np.argmax(ys_test_onehot, axis=1)
# easier to remember handle
correct_classes = ys_test
n = len(ys_test)

In [16]:
cnn_model = CNN_MNIST(isolate=True) # isolate sessions

In [10]:
svm_model = SVM_MNIST(mnist_train_features, ys_train, nb_classes=10)

In [12]:
hybrid_models = []
for tol in np.arange(0, 2.0, 0.5):
    hybrid_models.append(Hybrid_MNIST(accept_cnn_tolerance = tol))
for tol in np.arange(0, 2.0, 0.5):
    hybrid_models.append(Hybrid_MNIST(accept_cnn_tolerance = tol, stronger_criterion=True))

In [13]:
# instantiate models we want to test
# only one at a time here to avoid interference

gp_model = GP_MNIST_SVGP(kernel=gpflow.kernels.Linear, 
                         whitevar=0.1,
                         latent_split=20,
                         whiten=True,
                         q_diag=True,
                         minibatch=8000,
                         whitevar_trainable=False,
                         feat_trainable=True,
                         name="Linear Kernel GP")

# gp_model = GP_MNIST_SVGP(kernel=gpflow.kernels.Matern12, 
#                          whitevar=0.1,
#                          latent_split=20,
#                          whiten=True,
#                          q_diag=True,
#                          minibatch=8000,
#                          whitevar_trainable=False,
#                          feat_trainable=True,
#                          name="Matern12 Kernel GP")

# gp_model = GP_MNIST_SVGP(kernel=gpflow.kernels.Matern32, 
#                          whitevar=0.1,
#                          latent_split=20,
#                          whiten=True,
#                          q_diag=True,
#                          minibatch=8000,
#                          whitevar_trainable=False,
#                          feat_trainable=True,
#                          name="Matern32 Kernel GP")

# gp_model = GP_MNIST_SVGP(kernel=gpflow.kernels.Matern32, 
#                          whitevar=0.1,
#                          latent_split=20,
#                          whiten=True,
#                          q_diag=True,
#                          minibatch=8000,
#                          whitevar_trainable=False,
#                          feat_trainable=True,
#                          name="Matern52 Kernel GP")

# gp_model = GP_MNIST_SVGP(kernel=gpflow.kernels.RBF, 
#                          whitevar=0.1,
#                          latent_split=20,
#                          whiten=True,
#                          q_diag=True,
#                          minibatch=8000,
#                          whitevar_trainable=False,
#                          feat_trainable=True,
#                          name="RBF Kernel GP")


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4515.149993
  Number of iterations: 11
  Number of functions evaluations: 20


In [17]:
record_in = 'Standard MNIST'

# prediction accuracies
subkey = 'accuracy'

#CNN
mnist_cnn_preds = cnn_model.predict_img_batch(mnist_test_imgs)
mnist_cnn_classes = np.argmax(mnist_cnn_preds, axis=1)
mnist_cnn_correct = correct_classes = mnist_cnn_classes
data[record_in][subkey][cnn_model.name] = mnist_cnn_correct / n

# SVM
mnist_svm_classes = svm_model.predict_batch(mnist_test_features)
mnist_svm_correct = correct_classes = mnist_svm_classes
data[record_int][subkey][svm_model.name] = mnist_svm_correct / n

ValueError: Error when checking : expected conv2d_1_input to have 4 dimensions, but got array with shape (10000, 28, 28)

In [ ]:
# GP
mnist_gp_mu, mnist_gp_var = gp_model.predict_batch(mnist_test_features)
mnist_gp_classes = np.argmax(mnist_gp_mu, axis=1)
mnist_gp_correct = correct_classes = mnist_gp_classes
data[record_in][subkey][gp_model.name] = mnist_cnn_correct / n

for hybrid in hybrid_models:
    name = hybrid.name + "(GP: {})".format(gp_model.name)
    hybrid_pred, hybrid_mus, hybrid_vars = combined_predict_efficient(mnist_cnn_preds, mnist_gp_mu, mnist_gp_var)
    hybrid_classes = hybrid_pred[:, 1]
    hybrid_correct = correct_classes = hybrid_classes
    data[record_in][subkey][name] = hybrid_correct / n

In [ ]:
# At this point we can settle on one or two GP + 4 hybrid variants

# Preprocess the N-MNIST data just like standard MNIST
nmnist_noisy = scipy.io.loadmat('../data/raw/n-mnist/nmnist-awgn.mat')

record_in = 'MNIST + White noise'
if record_in not in data:
    data[record_in] = {} 
# noisy
subkey = 'Accuracy'
if subkey not in data[record_in]:
    data[record_in][subkey] = {}
    

noisy_feats = mnist_cnn.extract_features(nmnist_noisy)

In [ ]:
nmnist_blur = scipy.io.loadmat('../data/raw/n-mnist/nmnist-blur.mat')
record_in = 'Blurry MNIST'
if record_in not in data:
    data[record_in] = {} 
# noisy
subkey = 'Accuracy'
if subkey not in data[record_in]:
    data[record_in][subkey] = {}
    
blur_feats = mnist_cnn.extract_features(nmnist_blur)

In [ ]:
nmnist_noisy_lowcontrast = scipy.io.loadmat('../data/raw/n-mnist/nmnist-contrast.mat')
record_in = 'Low Contrast MNIST'
if record_in not in data:
    data[record_in] = {} 
# noisy
subkey = 'Accuracy'
if subkey not in data[record_in]:
    data[record_in][subkey] = {}
    
lowcontrast_feats = mnist_cnn.extract_features(nmnist_noisy_lowcontrast)